# 📊 Análisis Inicial de Datos de Movilidad Laboral

Este cuaderno realiza una primera exploración del dataset de trabajadores, sus residencias y lugares de trabajo. Incluye limpieza, geocodificación, cálculo de distancias y una estimación preliminar de la huella de carbono.

In [ ]:
# Cargar librerías necesarias
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time

In [ ]:
# Cargar los datos desde un archivo Excel o CSV
# Sustituir 'ruta/datos.xlsx' por la ruta real
df = pd.read_excel('data/raw/datos_movilidad.xlsx')
df.head()

## 🧹 Limpieza y revisión de datos

In [ ]:
# Revisamos columnas nulas, duplicadas, etc.
print(df.info())
print(df.isnull().sum())

## 🌍 Geocodificación de direcciones

In [ ]:
# Inicializamos el geocodificador (Nominatim)
geolocator = Nominatim(user_agent='movilidad_laboral')

def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
    except:
        pass
    return None, None

In [ ]:
# Aplicar geocodificación a residencia y trabajo
df['Coords_Residencia'] = df['Residencia'].apply(geocode_address)
time.sleep(1)
df['Coords_Trabajo'] = df['Direccion_Trabajo'].apply(geocode_address)
time.sleep(1)

# Separar lat/lon
df[['Lat_Residencia', 'Lon_Residencia']] = pd.DataFrame(df['Coords_Residencia'].tolist(), index=df.index)
df[['Lat_Trabajo', 'Lon_Trabajo']] = pd.DataFrame(df['Coords_Trabajo'].tolist(), index=df.index)

## 📏 Cálculo de distancia (km) entre residencia y trabajo

In [ ]:
def calcular_distancia(row):
    origen = (row['Lat_Residencia'], row['Lon_Residencia'])
    destino = (row['Lat_Trabajo'], row['Lon_Trabajo'])
    if None in origen or None in destino:
        return None
    return geodesic(origen, destino).km

df['Distancia_km'] = df.apply(calcular_distancia, axis=1)

## ♻️ Estimación de emisiones de CO₂

In [ ]:
# Suposición: modo de transporte en función de la distancia
def estimar_transporte(dist_km):
    if dist_km is None:
        return 'desconocido'
    elif dist_km < 3:
        return 'a pie'
    elif dist_km < 10:
        return 'bicicleta'
    elif dist_km < 40:
        return 'coche'
    else:
        return 'transporte público'

df['Modo_Transporte'] = df['Distancia_km'].apply(estimar_transporte)

# Emisiones por km (valores aproximados)
emisiones_por_km = {
    'a pie': 0,
    'bicicleta': 0,
    'coche': 0.21,
    'transporte público': 0.11,
    'desconocido': 0
}

df['Emision_dia_kgCO2'] = df.apply(lambda row: row['Distancia_km'] * emisiones_por_km.get(row['Modo_Transporte'], 0), axis=1)
df['Emision_anual_kgCO2'] = df['Emision_dia_kgCO2'] * 220  # Suponiendo 220 días laborales

In [ ]:
# Visualizamos los resultados
df[['Residencia', 'Direccion_Trabajo', 'Distancia_km', 'Modo_Transporte', 'Emision_dia_kgCO2', 'Emision_anual_kgCO2']].head()